# Curso Prático de Jurimetria - Recuperação Judicial

## Notebook 01: Análise Exploratória de Dados

Bem-vindo ao primeiro notebook do curso! Aqui você vai aprender a:

1. Carregar dados consolidados
2. Explorar estatísticas descritivas
3. Visualizar distribuições de flags
4. Identificar padrões em processos de recuperação judicial

## 📚 Importar Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✅ Bibliotecas importadas com sucesso!')

## 📂 Carregar Dados Consolidados

Vamos carregar o arquivo CSV gerado pelo pipeline.

In [ ]:
# Definir caminho do arquivo
data_path = Path('../outputs/consolidado_flags.csv')

# Verificar se o arquivo existe
if not data_path.exists():
    print('⚠️  Arquivo não encontrado!')
    print('Execute primeiro: python starter_scripts/01_pipeline_responder_14_questoes.py')
else:
    # Carregar dados
    df = pd.read_csv(data_path)
    print(f'✅ Dados carregados: {len(df)} processos')
    print(f'📊 Colunas: {df.shape[1]}')

## 🔍 Primeiras Linhas dos Dados

In [ ]:
# Visualizar primeiras linhas
df.head()

## 📊 Estatísticas Descritivas

In [ ]:
# Informações gerais
df.info()

In [ ]:
# Estatísticas de colunas numéricas
df.describe()

## 🏁 Análise das Flags

Vamos analisar as flags booleanas que identificam características dos processos.

In [ ]:
# Identificar colunas de flags (booleanas)
flag_columns = [
    'tem_recurso', 'tem_agravo', 'tem_apelacao', 'tem_embargos',
    'tem_liminar', 'tem_decisao_monocrática', 'tem_sentenca', 'tem_acordao',
    'processo_ativo', 'processo_arquivado', 'processo_suspenso',
    'valor_causa_alto', 'valor_causa_baixo', 'tempo_tramitacao_longo'
]

# Filtrar apenas flags que existem no DataFrame
existing_flags = [col for col in flag_columns if col in df.columns]

# Contar ocorrências de cada flag
flag_counts = df[existing_flags].sum().sort_values(ascending=False)

print('\n📈 Contagem de Flags:')
print('=' * 50)
for flag, count in flag_counts.items():
    percentage = (count / len(df)) * 100
    print(f'{flag:30s}: {count:4d} ({percentage:5.1f}%)')

## 📊 Visualização: Distribuição das Flags

In [ ]:
# Criar gráfico de barras
plt.figure(figsize=(12, 8))
flag_counts.plot(kind='barh', color='steelblue')
plt.title('Distribuição das Flags nos Processos de Recuperação Judicial', 
          fontsize=14, fontweight='bold')
plt.xlabel('Quantidade de Processos', fontsize=12)
plt.ylabel('Flag', fontsize=12)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 💰 Análise do Valor da Causa

In [ ]:
if 'valor_causa' in df.columns:
    # Remover valores nulos
    valores = df['valor_causa'].dropna()
    
    if len(valores) > 0:
        print(f'\n💰 Estatísticas do Valor da Causa:')
        print('=' * 50)
        print(f'Média: R$ {valores.mean():,.2f}')
        print(f'Mediana: R$ {valores.median():,.2f}')
        print(f'Mínimo: R$ {valores.min():,.2f}')
        print(f'Máximo: R$ {valores.max():,.2f}')
        print(f'Desvio Padrão: R$ {valores.std():,.2f}')
        
        # Visualização
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Histograma
        axes[0].hist(valores, bins=20, color='skyblue', edgecolor='black')
        axes[0].set_title('Distribuição do Valor da Causa', fontweight='bold')
        axes[0].set_xlabel('Valor (R$)')
        axes[0].set_ylabel('Frequência')
        axes[0].grid(alpha=0.3)
        
        # Boxplot
        axes[1].boxplot(valores, vert=True)
        axes[1].set_title('Boxplot do Valor da Causa', fontweight='bold')
        axes[1].set_ylabel('Valor (R$)')
        axes[1].grid(alpha=0.3)
        
        plt.tight_layout()
        plt.show()
else:
    print('⚠️  Coluna "valor_causa" não encontrada')

## ⏱️ Análise do Tempo de Tramitação

In [ ]:
if 'data_distribuicao' in df.columns:
    # Converter para datetime
    df['data_distribuicao'] = pd.to_datetime(df['data_distribuicao'], errors='coerce')
    
    # Calcular dias de tramitação
    df['dias_tramitacao'] = (pd.Timestamp.now() - df['data_distribuicao']).dt.days
    
    # Remover valores inválidos
    dias_validos = df['dias_tramitacao'].dropna()
    
    if len(dias_validos) > 0:
        print(f'\n⏱️  Estatísticas do Tempo de Tramitação:')
        print('=' * 50)
        print(f'Média: {dias_validos.mean():.0f} dias')
        print(f'Mediana: {dias_validos.median():.0f} dias')
        print(f'Mínimo: {dias_validos.min():.0f} dias')
        print(f'Máximo: {dias_validos.max():.0f} dias')
        
        # Visualização
        plt.figure(figsize=(10, 6))
        plt.hist(dias_validos, bins=30, color='coral', edgecolor='black')
        plt.title('Distribuição do Tempo de Tramitação', fontsize=14, fontweight='bold')
        plt.xlabel('Dias')
        plt.ylabel('Frequência')
        plt.axvline(dias_validos.mean(), color='red', linestyle='--', label=f'Média: {dias_validos.mean():.0f} dias')
        plt.axvline(dias_validos.median(), color='green', linestyle='--', label=f'Mediana: {dias_validos.median():.0f} dias')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print('⚠️  Coluna "data_distribuicao" não encontrada')

## 🔗 Correlação entre Flags

Vamos verificar se há correlação entre diferentes flags.

In [ ]:
if len(existing_flags) > 1:
    # Calcular matriz de correlação
    correlation_matrix = df[existing_flags].astype(int).corr()
    
    # Visualizar heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Matriz de Correlação entre Flags', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print('⚠️  Flags insuficientes para análise de correlação')

## 💡 Próximos Passos

Após esta análise exploratória, você pode:

1. **Adicionar mais dados** na pasta `data/` e executar o pipeline novamente
2. **Criar modelos preditivos** para classificar resultados de recuperações
3. **Análise de sobrevivência** (Kaplan-Meier, Cox) para tempo até decisão
4. **Web scraping** para coletar mais dados de tribunais
5. **Dashboard interativo** com Dash ou Streamlit

---

**Parabéns!** 🎉 Você completou o primeiro notebook do curso de jurimetria.

## 📝 Exercícios

1. Crie uma nova flag personalizada baseada em critérios próprios
2. Calcule a proporção de processos com valor de causa acima da mediana
3. Identifique processos que estão tramitando há mais de 3 anos
4. Crie um gráfico de pizza mostrando a distribuição de situações dos processos